<a href="https://colab.research.google.com/github/KhushiBhambri/YogNet/blob/main/PoseStreamModel/FinalPoseStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# RGBmodel = load_model('RGBStream.h5')
# Posemodel = load_model('PoseStream.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Mounted at /content/drive
/content/drive/MyDrive/Major_Project_Cse


In [ ]:
!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8,model_complexity=2)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def mpPoseImg(image_pose, pose=pose_image, draw=False, display=False):
    
    annotated_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
 
    
    results = pose.process(image_in_RGB)

    if results.pose_landmarks and draw:    
      mp_drawing.draw_landmarks(image=annotated_image,landmark_list=results.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        # mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
        #                           landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),thickness=3, circle_radius=3),
        #                           connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=2, circle_radius=2))

    if display:
            
            # plt.figure(figsize=[22,22])
            # plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            # plt.subplot(122);plt.imshow(annotated_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
            cv2.imshow(annotated_image)
            return results

    else:
        return annotated_image,results
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 47.1 MB/s eta 0:00:00


In [ ]:
# calculate the center of the object in each frame
def CentralisedKeypoints1(keypoints):
  frame_centers = []
 
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))

  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no
    for k in range(len(relative_keypoints[j])):          # each keypoint of frame
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

  frame=0
  for i in range(len(relative_keypoints)):   #each frame
    countx=0
    county=0
    for j in range(len(relative_keypoints[i])):   #34 keypoints
      if relative_keypoints[i][j][0] is None:
        relative_keypoints[i][j][0] = -1
        countx+=1
      if relative_keypoints[i][j][1] is None:
        relative_keypoints[i][j][1] = -1
        county+=1
    if(countx== 34 and county==34):
       frame+=1
  if(frame>0): 
    return None
  return relative_keypoints



In [ ]:
numOfKeypoints=34

import numpy as np
import tensorflow as tf

# calculate the center of the object in each frame

def CentralisedKeypoints2(keypoints):
  frame_centers = []
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))


  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no j
    for k in range(len(relative_keypoints[j])):          # k keypoint of frame j
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

    frames_keypoints=[]
    for j in range(len(relative_keypoints)):
        flag=1
        for k in range(len(relative_keypoints[j])):
          if relative_keypoints[j][k][0] is None or relative_keypoints[j][k][1] is None:
            flag=0
            break
        if(flag==1):
          frames_keypoints.append(relative_keypoints[j])

    if(len(frames_keypoints)>0):
      kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
      for x in range (16-len(frames_keypoints)):
            frames_keypoints.append(kk)
      return frames_keypoints

  return None



In [ ]:
numOfKeypoints = 34
def getKeyPointsforFrame(frame):
  img,results = mpPoseImg(frame)
  keypoints = [[None,None] for _ in range(numOfKeypoints)]
  if results.pose_landmarks == None:
    return keypoints
    
  landmarks = results.pose_landmarks
  for i in range(numOfKeypoints-2):
     mark = landmarks.landmark[i]
     keypoints[i][0]=mark.x
     keypoints[i][1]=mark.y
  #------------------------------------------------------------------------------------------------
  try:
    # MID HIP COORDINATES
      left_hip_index = 23
      right_hip_index = 24
      Mid_hip_x =  (keypoints[left_hip_index][0]+keypoints[right_hip_index][0])/2
      Mid_hip_y =(keypoints[left_hip_index][1]+keypoints[right_hip_index][1])/2
      # keypoints.append([Mid_hip_x,Mid_hip_y])
      keypoints[32][0]=Mid_hip_x
      keypoints[32][1]=Mid_hip_y
                          
  except:
      pass
      
  #------------------------------------------------------------------------------------------------
  try:
    # NECK COORDINATES
      left_shoulder_index = 11
      right_shoulder_index = 12
      Neck_x =  (keypoints[left_shoulder_index][0]+keypoints[right_shoulder_index][0])/2
      Neck_y =(keypoints[left_shoulder_index][1]+keypoints[right_shoulder_index][1])/2
      # keypoints.append([Neck_x,Neck_y])
      keypoints[33][0]=Neck_x
      keypoints[33][1]=Neck_y
                          
  except:
      pass

  #------------------------------------------------------------------------------------------------
     
  return keypoints

In [ ]:
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model



In [ ]:
import pandas as pd

# read the data from the CSV file
data = pd.read_csv('train_data.csv')

# load the keypoints and labels into separate variables
# train_keypoints = list(data['keypoints'])
train_keypoints=[]
train_labels = list(data['label'])

numOfKeypoints = 34
import ast
for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [ ]:
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_keypoints)):
  keypoints = CentralisedKeypoints1(train_keypoints[i])
  if(keypoints is not None):
     new_train_keypoints.append(keypoints)
     new_train_labels.append(train_labels[i])


In [ ]:
len(new_train_keypoints)

916

In [ ]:
# train_videos = np.asarray(train_videos)
train_keypoints = np.asarray(train_keypoints)
train_labels = np.asarray(train_labels).astype('int32')

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(train_keypoints,train_labels,test_size=0.25,shuffle=True)

In [ ]:
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300
23/23 [==============================] - 2s 102ms/step - loss: 21.1246 - accuracy: 0.0977 - val_loss: 17.1037 - val_accuracy: 0.1484
Epoch 2/300
23/23 [==============================] - 2s 108ms/step - loss: 14.1230 - accuracy: 0.1664 - val_loss: 11.1578 - val_accuracy: 0.0549
Epoch 3/300
23/23 [==============================] - 2s 77ms/step - loss: 8.9906 - accuracy: 0.1829 - val_loss: 7.0700 - val_accuracy: 0.2308
Epoch 4/300
23/23 [==============================] - 2s 82ms/step - loss: 5.8047 - accuracy: 0.2600 - val_loss: 4.7437 - val_accuracy: 0.2143
Epoch 5/300
23/23 [==============================] - 2s 83ms/step - loss: 4.0726 - accuracy: 0.2627 - val_loss: 3.4800 - val_accuracy: 0.3297
Epoch 6/300
23/23 [==============================] - 2s 84ms/step - loss: 3.1694 - accuracy: 0.3453 - val_loss: 2.9011 - val_accuracy: 0.3626
Epoch 7/300
23/23 [==============================] - 2s 83ms/step - loss: 2.8978 - accuracy: 0.3796 - val_loss: 2.7390 - val_accuracy: 0.4176


In [ ]:
model.evaluate(XKey_test,yKey_test)

10/10 [==============================] - 0s 46ms/step - loss: 1.0725 - accuracy: 0.7862


[1.0725456476211548, 0.7861841917037964]

In [ ]:
# Save the model to a file
model.save('PoseStream1.h5')

In [ ]:
softmaxRGB = RGBmodel.predict(X_trainRGB)
softmaxPose = Posemodel.predict(XKey_train)

In [ ]:
import numpy as np

# Combine the softmax outputs by taking the average of their values
combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

# Generate some example ground truth labels
actualValues = Ytrain

# Convert the combined softmax probabilities to predicted labels
predicted_labels = np.argmax(combined_softmax)

# Compute the accuracy of the predictions
accuracy = np.mean(predicted_labels == actualValues)

# Print the results
print("Combined softmax probabilities:", combined_softmax)
print("Ground truth labels:", actualValues)
print("Predicted labels:", predicted_labels)
print("Accuracy:", accuracy)


In [ ]:
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = '../MyDrive/MyDrive/Dataset'


labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107

train_videos=[]
train_labels=[]
train_keypoints = []
import traceback
import random

num = 1

for i,folder in enumerate(labels):
  try:
    for video in os.listdir(data_path+'/'+folder):
      print(num)
      num+=1
      video = os.path.join(data_path+'/'+folder+'/'+video)
      # Open the video file
      cap = cv2.VideoCapture(video)

      # Get the total number of frames in the video
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      # Select a random starting frame number
      start_frame = random.randint(0, total_frames - 16)

      # Set the number of frames to be selected
      num_frames = 16

      # Set the frame number to start with
      cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

      # Loop through the frames and save them
      frames = []
      keypoints = []
      for j in range(num_frames):
          ret, image = cap.read()
          if ret:
            frames.append(cv2.resize(image,(112,112)))
            kk = getKeyPointsforFrame(image)
            keypoints.append(kk)

      # Release the video file
      cap.release()


      train_videos.append(frames)

      train_keypoints.append(CentralisedKeypoints2(keypoints))
      train_labels.append(i)
      # if(num>2):
      #    break

  except Exception:
    traceback.print_exc()

In [ ]:
softmaxRGB = RGBmodel.predict(X_trainRGB)
softmaxPose = Posemodel.predict(XKey_train)

In [ ]:
import numpy as np

# Combine the softmax outputs by taking the average of their values
combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

# Generate some example ground truth labels
actualValues = yKey_train

# Convert the combined softmax probabilities to predicted labels
predicted_labels = np.argmax(combined_softmax)

# Compute the accuracy of the predictions
accuracy = np.mean(predicted_labels == actualValues)

# Print the results
print("Combined softmax probabilities:", combined_softmax)
print("Ground truth labels:", actualValues)
print("Predicted labels:", predicted_labels)
print("Accuracy:", accuracy)



In [ ]:
labels=['Adhomukhasvanasana',
  'Ardhachakrasana',
  'Bhujangasana',
  'Dhanurasana',
  'Marjariasana',
  'Padahastasana',
  'Padmasana',
  'Pawanmuktasana',
  'Phalakasana',
  'Sarvangasana',
  'Sashankasana',
  'Setubandhasana',
  'Shavasana',
  'Tadasana',
  'Trikonasana',
  'Ustrasana',
  'Vakrasana',
  'Virbhadrasana1',
  'Virbhadrasana2',
  'Vrikshasana']
  
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort()

def predictYogaPose(video,keypoints):
  softmaxRGB = RGBmodel.predict(video)
  softmaxPose = Posemodel.predict(keypoints)
  combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

  predicted_labels = np.argmax(combined_softmax)

  return labels[predicted_labels]




In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

data_path = './Videos'

labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107
labels

['Adhomukhasvanasana',
 'Ardhachakrasana',
 'Bhujangasana',
 'Dhanurasana',
 'Marjariasana',
 'Padahastasana',
 'Padmasana',
 'Pawanmuktasana',
 'Phalakasana',
 'Sarvangasana',
 'Sashankasana',
 'Setubandhasana',
 'Shavasana',
 'Tadasana',
 'Trikonasana',
 'Ustrasana',
 'Vakrasana',
 'Virbhadrasana1',
 'Virbhadrasana2',
 'Vrikshasana']

In [ ]:
RGBmodel = load_model('RGBStream.h5')
Posemodel = load_model('PoseStream.h5')

import cv2
cap = cv2.VideoCapture(0)

video = []

while True:
  ret, frame = cap.read()

  if not ret:
    break
  
  video.append(frame)

  if(len(video)==16):
    keypoints = []
    for image in video:
      keypoints.append(getKeyPointsforFrame(image))

    keypoints = CentralisedKeypoints2(keypoints)

    if(keypoints == None):
      print("video not visible")

    yogaPose = predictYogaPose(video,keypoints)

